## Using Schemas

In [4]:
SELECT * FROM public.customers LIMIT 5;

---
### Create a schema for customer objects and set up basic permissions

In [8]:
CREATE ROLE cust_owner;
CREATE SCHEMA cust AUTHORIZATION cust_owner;
GRANT USAGE ON SCHEMA cust to cust_owner;

ALTER DEFAULT PRIVILEGES 
    IN SCHEMA cust
    GRANT SELECT, INSERT, UPDATE, DELETE ON TABLES TO cust_owner;

---
### Copy customers table to new schema

In [9]:
CREATE TABLE cust.customers AS
    SELECT * FROM public.customers;

---
### Create function to read name from the customers table

In [2]:
CREATE OR REPLACE FUNCTION cust.get_cust_name(id INTEGER) 
    RETURNS VARCHAR
    LANGUAGE SQL
    STABLE LEAKPROOF
    SET search_path = cust
    SECURITY DEFINER
AS $$
    SELECT CONCAT_WS(' ', firstname, lastname) 
    FROM cust.customers c
    WHERE c.customerid = id;
$$;

ALTER FUNCTION cust.get_cust_name(id INTEGER)
    OWNER to cust_owner;

---
### Create report analyst role and users and revoke privileges

In [3]:
CREATE GROUP report_analysts;
CREATE USER Lindsay IN GROUP report_analysts PASSWORD 'p455w0rd';
GRANT USAGE ON SCHEMA cust TO report_analysts;

REVOKE SELECT, INSERT, UPDATE, DELETE 
    ON ALL TABLES IN SCHEMA cust FROM report_analysts;
GRANT EXECUTE ON ALL FUNCTIONS IN SCHEMA cust TO report_analysts;

---
### Clean Up

In [1]:
DROP FUNCTION IF EXISTS cust.get_cust_name(INT);
DROP TABLE IF EXISTS cust.customers;
DROP SCHEMA IF EXISTS cust;
DROP ROLE IF EXISTS cust_owner;
DROP USER IF EXISTS Lindsay;
DROP GROUP IF EXISTS report_analysts;